# Importar librerias

## Instalacion librerias

In [1]:
import pandas as pd
import re #Sirve para string

# Importación dataset

In [122]:
# definicion del dataframe
df = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
df.head(10)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277
5,231808869,04/04/2023 12:00:00 AM,12/01/2020 12:00:00 AM,2300,18,Southeast,1826,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,9900 COMPTON AV,NaN,33.9467,-118.2463
6,230110144,04/04/2023 12:00:00 AM,07/03/2020 12:00:00 AM,900,1,Central,182,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1100 S GRAND AV,NaN,34.0415,-118.2620
7,220314085,07/22/2022 12:00:00 AM,05/12/2020 12:00:00 AM,1110,3,Southwest,303,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,2500 S SYCAMORE AV,NaN,34.0335,-118.3537
8,231309864,04/28/2023 12:00:00 AM,12/09/2020 12:00:00 AM,1400,13,Newton,1375,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1300 E 57TH ST,NaN,33.9911,-118.2521
9,211904005,12/31/2020 12:00:00 AM,12/31/2020 12:00:00 AM,1220,19,Mission,1974,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,9000 CEDROS AV,NaN,34.2336,-118.4535


In [123]:
df.iloc[:,:14].head()


,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,NaN,0,M,O
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,M,O
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,0344 1251,19,X,X
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,M,O
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,M,H


In [124]:
df.iloc[:,14:].head()

,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,101.0,STREET,NaN,NaN,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,128.0,BUS STOP/LAYOVER (ALSO QUERY 124),NaN,NaN,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,405.0,CLOTHING STORE,NaN,NaN,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,102.0,SIDEWALK,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [125]:
df.iloc[:,14:].tail()

,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
986495,207.0,BAR/COCKTAIL/NIGHTCLUB,NaN,NaN,IC,Invest Cont,341.0,NaN,NaN,NaN,3700 WILSHIRE BL,NaN,34.0617,-118.3066
986496,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,4000 W 23RD ST,NaN,34.0362,-118.3284
986497,503.0,HOTEL,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,745.0,NaN,NaN,NaN,1300 W SUNSET BL,NaN,34.0685,-118.2460
986498,102.0,SIDEWALK,308.0,STICK,IC,Invest Cont,230.0,NaN,NaN,NaN,FLOWER ST,JEFFERSON BL,34.0215,-118.2868
986499,108.0,PARKING LOT,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,6900 VESPER AV,NaN,34.1961,-118.4510


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986500 entries, 0 to 986499
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           986500 non-null  int64  
 1   Date Rptd       986500 non-null  object 
 2   DATE OCC        986500 non-null  object 
 3   TIME OCC        986500 non-null  int64  
 4   AREA            986500 non-null  int64  
 5   AREA NAME       986500 non-null  object 
 6   Rpt Dist No     986500 non-null  int64  
 7   Part 1-2        986500 non-null  int64  
 8   Crm Cd          986500 non-null  int64  
 9   Crm Cd Desc     986500 non-null  object 
 10  Mocodes         840065 non-null  object 
 11  Vict Age        986500 non-null  int64  
 12  Vict Sex        846925 non-null  object 
 13  Vict Descent    846914 non-null  object 
 14  Premis Cd       986486 non-null  float64
 15  Premis Desc     985915 non-null  object 
 16  Weapon Used Cd  326368 non-null  float64
 17  Weapon Des

In [127]:
df.describe()

,DR_NO,TIME OCC,AREA,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON
count,9.865000e+05,986500.000000,986500.000000,986500.000000,986500.000000,986500.000000,986500.000000,986486.000000,326368.000000,986489.000000,68912.000000,2310.000000,64.00000,986500.000000,986500.000000
mean,2.198270e+08,1339.086392,10.698086,1116.245129,1.403666,500.782398,29.045177,306.054803,363.846008,500.538333,958.162091,984.198701,991.21875,33.996033,-118.083281
std,1.299240e+07,651.457178,6.108873,611.002343,0.490632,206.092913,21.976666,219.156166,123.699000,205.891829,110.250287,51.495991,27.06985,1.633543,5.661853
min,8.170000e+02,1.000000,1.000000,101.000000,1.000000,110.000000,-4.000000,101.000000,101.000000,110.000000,210.000000,310.000000,821.00000,0.000000,-118.667600
25%,2.106103e+08,900.000000,5.000000,587.000000,1.000000,331.000000,0.000000,101.000000,311.000000,331.000000,998.000000,998.000000,998.00000,34.014600,-118.430500
50%,2.208174e+08,1420.000000,11.000000,1141.000000,1.000000,442.000000,30.000000,203.000000,400.000000,442.000000,998.000000,998.000000,998.00000,34.058900,-118.322500
75%,2.310049e+08,1900.000000,16.000000,1617.000000,2.000000,626.000000,44.000000,501.000000,400.000000,626.000000,998.000000,998.000000,998.00000,34.164900,-118.273900
max,2.499289e+08,2359.000000,21.000000,2199.000000,2.000000,956.000000,120.000000,976.000000,516.000000,956.000000,999.000000,999.000000,999.00000,34.334300,0.000000


In [131]:
# Exploración de primeros 50 valores únicos solo para variables categóricas
for col in df.select_dtypes(include='object').columns:
    print(f"--- {col} ---")
    print(df[col].dropna().unique()[:50])  # Primeros 50 valores únicos
    print("\n")



--- date_report ---
['03/01/2020 12:00:00 AM' '02/09/2020 12:00:00 AM'
 '11/11/2020 12:00:00 AM' '05/10/2023 12:00:00 AM'
 '08/18/2022 12:00:00 AM' '04/04/2023 12:00:00 AM'
 '07/22/2022 12:00:00 AM' '04/28/2023 12:00:00 AM'
 '12/31/2020 12:00:00 AM' '01/21/2022 12:00:00 AM'
 '04/12/2022 12:00:00 AM' '01/05/2023 12:00:00 AM'
 '05/06/2022 12:00:00 AM' '06/01/2023 12:00:00 AM'
 '02/03/2023 12:00:00 AM' '12/24/2023 12:00:00 AM'
 '11/27/2020 12:00:00 AM' '09/20/2022 12:00:00 AM'
 '02/03/2022 12:00:00 AM' '04/25/2023 12:00:00 AM'
 '03/21/2023 12:00:00 AM' '06/04/2023 12:00:00 AM'
 '09/09/2020 12:00:00 AM' '08/03/2023 12:00:00 AM'
 '02/02/2022 12:00:00 AM' '02/23/2022 12:00:00 AM'
 '05/03/2020 12:00:00 AM' '03/20/2021 12:00:00 AM'
 '04/28/2022 12:00:00 AM' '02/27/2023 12:00:00 AM'
 '01/17/2023 12:00:00 AM' '05/09/2023 12:00:00 AM'
 '01/19/2022 12:00:00 AM' '09/06/2022 12:00:00 AM'
 '09/27/2021 12:00:00 AM' '10/22/2021 12:00:00 AM'
 '10/10/2022 12:00:00 AM' '03/28/2023 12:00:00 AM'
 '10/25/202

In [132]:
# Vamos a renombrar las columnas para hacerlas más comprensibles, se trata de un cambio en el Diccionario
rename_columns = {
    'DR_NO': 'id_report',
    'Date Rptd': 'date_report',
    'DATE OCC': 'date_ocurrance',
    'TIME OCC': 'time_ocurrance',
    'AREA': 'area_code',
    'AREA NAME': 'area_name',
    'Rpt Dist No': 'district',
    'Part 1-2': 'crime_severity',
    'Crm Cd': 'crime_code',
    'Crm Cd Desc': 'crime_desc',
    'Mocodes': 'crime_mo',
    'Vict Age': 'age_victim',
    'Vict Sex': 'sex_victim',
    'Vict Descent': 'descent_victim',
    'Premis Cd': 'loc_code',
    'Premis Desc': 'loc_desc',
    'Weapon Used Cd': 'weapon_code',
    'Weapon Desc': 'weapon_desc',
    'Status': 'case_status_code',
    'Status Desc': 'case_status_desc',
    'Crm Cd 1': 'crime_code_1',
    'Crm Cd 2': 'crime_code_2',
    'Crm Cd 3': 'crime_code_3',
    'Crm Cd 4': 'crime_code_4',
    'LOCATION': 'location',
    'Cross Street': 'cross_street',
    'LAT': 'latitude',
    'LON': 'longitude'
}

# Renombrar las columnas
df.rename(columns=rename_columns, inplace=True)

# Confirmar el cambio
df.head()


,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,case_status_code,case_status_desc,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [133]:
# Variables a explorar
columns_to_explore = [
    'area_code',
    'area_name',
    'district',
    'crime_severity',
    'crime_code',
    'crime_desc',
    'sex_victim',
    'descent_victim',
    'loc_code',
    'loc_desc',
    'weapon_code',
    'weapon_desc',
    'case_status_code',
    'case_status_desc'
]

# Mostrar TODOS los valores únicos para esas columnas
for col in columns_to_explore:
    if col in df.columns:
        print(f"--- {col} ---")
        print(sorted(df[col].dropna().unique()))
        print("\n")
    else:
        print(f"Column {col} not found in the dataframe.\n")


--- area_code ---
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21)]


--- area_name ---
['77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck', 'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire']


--- district ---
[np.int64(101), np.int64(105), np.int64(109), np.int64(111), np.int64(112), np.int64(118), np.int64(119), np.int64(121), np.int64(122), np.int64(123), np.int64(124), np.int64(127), np.int64(128), np.int64(129), np.int64(131), np.int64(132), np.int64(133), np.int64(134), np.int64(135), np.int64(138), np.int64(139), np.int64(141), np.int64(142), np.int64(143), np.int64(144), np.int64(145), np.int6

## **Vamos a trabajar en investigar la relación de códigos varios en el dataset. **

# 📄 Unique Values Summary

## area_code
`1, 2, 3, ..., 21`

## area_name
`'77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck', 'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire'`

## district
`101, 105, 109, 111, ..., 2199`  
(Muchos valores únicos de distrito, códigos de zonas específicas)

## crime_severity
`1, 2`

## crime_code
`110, 113, 121, 122, 210, 220, 230, 231, ..., 956`

## crime_desc
`'ARSON', 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER', 'BATTERY - SIMPLE ASSAULT', 'BIKE - STOLEN', 'BURGLARY', 'BURGLARY FROM VEHICLE', ..., 'WEAPONS POSSESSION/BOMBING'`
(Más de 100 tipos de crímenes diferentes)

## crime_mo
`Códigos tipo '0344 1822 1402', '0930', '0913 2024' etc.`  
(Modus Operandi combinados por código)

## sex_victim
`'-', 'F', 'H', 'M', 'X'`
(Indica sexo: Femenino, Masculino, Indeterminado, No reportado)

## descent_victim
`'-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z'`
(Origen étnico de la víctima)

## loc_code
`101.0, 102.0, 103.0, ..., 976.0`
(Muchos códigos de localización)

## loc_desc
`'STREET', 'BUS STOP', 'PARKING LOT', 'SINGLE FAMILY DWELLING', 'SCHOOL', 'HOSPITAL', 'HOTEL', 'MARKET', ..., 'YARD (RESIDENTIAL/BUSINESS)'`
(Lugares donde ocurrieron los crímenes)

## weapon_code
`101.0, 102.0, ..., 516.0`

## weapon_desc
`'HAND GUN', 'KNIFE WITH BLADE 6INCHES OR LESS', 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)', 'UNKNOWN FIREARM', 'VERBAL THREAT', ...`
(Descripción de armas usadas)

## case_status_code
`'AA', 'AO', 'CC', 'IC', 'JA', 'JO'`
(Códigos de estatus del caso)

## case_status_desc
`'Adult Arrest', 'Adult Other', 'Invest Cont', 'Juv Arrest', 'Juv Other', 'UNK'`
(Descripción del estatus del caso)


## 1. Investigación de las áreas.

In [134]:
# Primero aseguramos que district sea string para extraer area_code
df['district'] = df['district'].astype(str)

# Sacamos el primer dígito como área (puedes usar 1 o 2 dígitos según precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Mapeo de area_code a area_name
area_mapping = {
    1: '77th Street',
    2: 'Central',
    3: 'Devonshire',
    4: 'Foothill',
    5: 'Harbor',
    6: 'Hollenbeck',
    7: 'Hollywood',
    8: 'Mission',
    9: 'N Hollywood',
    10: 'Newton',
    11: 'Northeast',
    12: 'Olympic',
    13: 'Pacific',
    14: 'Rampart',
    15: 'Southeast',
    16: 'Southwest',
    17: 'Topanga',
    18: 'Van Nuys',
    19: 'West LA',
    20: 'West Valley',
    21: 'Wilshire'
}

# Asignamos nombre al área
df['area_name'] = df['area_code'].map(area_mapping)

# Verificamos si el mapeo está correcto
print(df[['district', 'area_code', 'area_name']].head())

# Contar registros por area_name
print(df['area_name'].value_counts())


  district  area_code    area_name
0      784          7    Hollywood
1      182          1  77th Street
2      356          3   Devonshire
3      964          9  N Hollywood
4      666          6   Hollenbeck
area_name
77th Street    531859
Central        135803
Devonshire      56259
Hollenbeck      51510
Hollywood       47272
Mission         44848
N Hollywood     42059
Harbor          40603
Foothill        36287
Name: count, dtype: int64


In [135]:

# Definimos el listado completo de áreas (en el orden correcto)
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# Hacemos el conteo de registros por area_name
area_counts = df['area_name'].value_counts()

# Aseguramos que todos los nombres estén incluidos (incluso los que faltan)
area_counts = area_counts.reindex(area_names, fill_value=0)

# Ahora, buscamos las áreas con 0 crímenes
areas_sin_crimen = area_counts[area_counts == 0]

# Mostrar áreas sin crímenes
print("Áreas donde no hubo crímenes registrados:")
print(areas_sin_crimen)

Áreas donde no hubo crímenes registrados:
area_name
Newton         0
Northeast      0
Olympic        0
Pacific        0
Rampart        0
Southeast      0
Southwest      0
Topanga        0
Van Nuys       0
West LA        0
West Valley    0
Wilshire       0
Name: count, dtype: int64


In [136]:
# Definimos las 21 áreas oficiales
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# 1. Conteo de crímenes por área
area_counts = df['area_name'].value_counts()

# 2. Asegurarnos que todas las áreas estén (rellenar con 0 si no existen)
area_counts = area_counts.reindex(area_names, fill_value=0)

# 3. Detectar áreas sin crímenes
areas_sin_crimen = area_counts[area_counts == 0].index.tolist()

print("Áreas donde no hubo crímenes:")
print(areas_sin_crimen)

# 4. Filtrar el df para filas con esas áreas (debería estar vacío, o muy pequeño si algo raro pasó)
df_sin_crimenes = df[df['area_name'].isin(areas_sin_crimen)]

# 5. Mostrar las filas
print(df_sin_crimenes)


Áreas donde no hubo crímenes:
['Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire']
Empty DataFrame
Columns: [id_report, date_report, date_ocurrance, time_ocurrance, area_code, area_name, district, crime_severity, crime_code, crime_desc, crime_mo, age_victim, sex_victim, descent_victim, loc_code, loc_desc, weapon_code, weapon_desc, case_status_code, case_status_desc, crime_code_1, crime_code_2, crime_code_3, crime_code_4, location, cross_street, latitude, longitude]
Index: []

[0 rows x 28 columns]


Resumen:
9 áreas sí tienen crímenes (las que viste antes: 77th Street, Central, Devonshire, etc.).

12 áreas no tienen crímenes en tus datos actuales.

No hay errores de mapeo en district, area_code ni area_name.

## 2. Investigación Distritos.

In [137]:
# Mostrar los valores únicos de la columna 'district'
if 'district' in df.columns:
    print("--- district ---")
    print(sorted(df['district'].dropna().unique()))
    print("\n")
else:
    print("Column 'district' not found in the dataframe.\n")

--- district ---
['1000', '1001', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '101', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '1021', '1022', '1023', '1024', '1025', '1027', '1028', '1029', '1031', '1032', '1033', '1034', '1035', '1036', '1037', '1039', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '105', '1053', '1061', '1063', '1065', '1067', '1068', '1075', '1077', '1078', '1079', '1083', '1087', '1088', '1089', '109', '1091', '1094', '1097', '1098', '1099', '1100', '1101', '1102', '1105', '1107', '1109', '111', '1112', '1113', '1115', '1116', '1117', '1118', '1119', '112', '1122', '1123', '1124', '1125', '1126', '1127', '1128', '1129', '1132', '1133', '1134', '1135', '1136', '1137', '1138', '1139', '1141', '1142', '1143', '1144', '1145', '1146', '1147', '1148', '1149', '1151', '1152', '1153', '1157', '1158', '1159', '1161', '1162', '1163', '1169', '1171', '1172', '1173', '1174', '1175', '1176', '1177', '1178', '11

In [138]:
# Asegurar que 'district' es string para extraer prefijos fácilmente
df['district'] = df['district'].astype(str)

# Extraer primer dígito (o primeros dos dígitos si quieres más precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Opcionalmente, si quieres extraer dos primeros dígitos:
# df['area_code'] = df['district'].str[:2].astype(int)

# Definir los áreas válidas (1 al 21)
valid_areas = list(range(1, 22))

# Marcar si es un área válida
df['area_valid'] = df['area_code'].isin(valid_areas)

# Verificar resultados
print(df['area_valid'].value_counts())

# Ver ejemplos
print(df[['district', 'area_code', 'area_valid']].head())


area_valid
True    986500
Name: count, dtype: int64
  district  area_code  area_valid
0      784          7        True
1      182          1        True
2      356          3        True
3      964          9        True
4      666          6        True


In [139]:
# Contar cuántos registros NO son válidos
invalid_count = (~df['area_valid']).sum()

print(f"Cantidad de registros con area_valid == False: {invalid_count}")


Cantidad de registros con area_valid == False: 0


## 3. Relación entre crime_code y crime_desc

Los crime_code y crime_desc que tienes provienen directamente de los reportes oficiales del LAPD (Los Angeles Police Department).

Cada "crime_code" es un tipo de crimen específico, basado en su severidad y en las leyes de California.

No es una clasificación inventada, es estándar dentro del departamento, usado para categorizar las denuncias.

In [140]:
# Comprobar si un mismo crime_code tiene varias descripciones
crime_mapping = df[['crime_code', 'crime_desc']].drop_duplicates()

# Verificar duplicados de crime_code
duplicates = crime_mapping.duplicated(subset=['crime_code'], keep=False)
crime_mapping[duplicates]

,crime_code,crime_desc


Si no salen filas, la relación es uno a uno ✅.

In [141]:
# Crear tabla resumen de códigos de crimen y su descripción
crime_table = df[['crime_code', 'crime_desc']].drop_duplicates().sort_values('crime_code')

# Mostrar la tabla
crime_table.reset_index(drop=True, inplace=True)
crime_table


,crime_code,crime_desc
0,110,CRIMINAL HOMICIDE
1,113,"MANSLAUGHTER, NEGLIGENT"
2,121,"RAPE, FORCIBLE"
3,122,"RAPE, ATTEMPTED"
4,210,ROBBERY
...,...,...
135,949,ILLEGAL DUMPING
136,950,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $..."
137,951,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &..."
138,954,CONTRIBUTING


In [20]:
# Pasar a csv la descripcion de los crime_code
#crime_table.to_csv('crime_code_description.csv', index=False)

Vamos a generar una agrupación de todos estos delitos en base al standard UCR (Uniform Crime Reporting)

In [142]:
# Suponemos que tienes un DataFrame llamado 'df' con las columnas 'crime_code' y 'crime_desc'

# Mapeo de crime_codes a categorías UCR
ucr_mapping = {
    # Part I - Violent Crimes
    110: 'HOMICIDE',
    113: 'MANSLAUGHTER',
    121: 'RAPE',
    122: 'ATTEMPTED RAPE',
    815: 'SEXUAL PENETRATION',
    820: 'ORAL COPULATION',
    821: 'SODOMY',
    210: 'ROBBERY',
    220: 'ROBBERY ATTEMPTED',
    230: 'AGGRAVATED ASSAULT',
    231: 'AGGRAVATED ASSAULT',
    235: 'AGGRAVATED ASSAULT',
    236: 'AGGRAVATED ASSAULT',
    250: 'AGGRAVATED ASSAULT',
    251: 'AGGRAVATED ASSAULT',
    761: 'AGGRAVATED ASSAULT',
    926: 'AGGRAVATED ASSAULT',
    435: 'SIMPLE ASSAULT',
    436: 'SIMPLE ASSAULT',
    437: 'SIMPLE ASSAULT',
    622: 'SIMPLE ASSAULT',
    623: 'SIMPLE ASSAULT',
    624: 'SIMPLE ASSAULT',
    625: 'SIMPLE ASSAULT',
    626: 'SIMPLE ASSAULT',
    627: 'SIMPLE ASSAULT',
    647: 'SIMPLE ASSAULT',
    763: 'SIMPLE ASSAULT',
    928: 'SIMPLE ASSAULT',
    930: 'SIMPLE ASSAULT',

    # Part I - Property Crimes
    310: 'BURGLARY',
    320: 'BURGLARY',
    510: 'MOTOR VEHICLE THEFT',
    520: 'MOTOR VEHICLE THEFT',
    433: 'MOTOR VEHICLE THEFT',
    330: 'THEFT FROM VEHICLE',
    331: 'THEFT FROM VEHICLE',
    410: 'THEFT FROM VEHICLE',
    420: 'THEFT FROM VEHICLE',
    421: 'THEFT FROM VEHICLE',
    350: 'PERSONAL THEFT',
    351: 'PERSONAL THEFT',
    352: 'PERSONAL THEFT',
    353: 'PERSONAL THEFT',
    450: 'PERSONAL THEFT',
    451: 'PERSONAL THEFT',
    452: 'PERSONAL THEFT',
    453: 'PERSONAL THEFT',
    341: 'OTHER THEFT',
    343: 'OTHER THEFT',
    345: 'OTHER THEFT',
    440: 'OTHER THEFT',
    441: 'OTHER THEFT',
    442: 'OTHER THEFT',
    443: 'OTHER THEFT',
    444: 'OTHER THEFT',
    445: 'OTHER THEFT',
    470: 'OTHER THEFT',
    471: 'OTHER THEFT',
    472: 'OTHER THEFT',
    473: 'OTHER THEFT',
    474: 'OTHER THEFT',
    475: 'OTHER THEFT',
    480: 'OTHER THEFT',
    485: 'OTHER THEFT',
    487: 'OTHER THEFT',
    491: 'OTHER THEFT',

    # Part II - Other Crimes
    500: 'DRUG OFFENSES',
    501: 'DRUG OFFENSES',
    502: 'DRUG OFFENSES',
    503: 'DRUG OFFENSES',
    504: 'DRUG OFFENSES',
    505: 'DRUG OFFENSES',
    506: 'DRUG OFFENSES',
    600: 'SIMPLE ASSAULT',
    601: 'SIMPLE ASSAULT',
    602: 'SIMPLE ASSAULT',
    603: 'SIMPLE ASSAULT',
    700: 'VANDALISM',
    701: 'VANDALISM',
    702: 'WEAPONS OFFENSE',
    703: 'WEAPONS OFFENSE',
    704: 'WEAPONS OFFENSE',
    705: 'WEAPONS OFFENSE',
    800: 'SEX OFFENSES (NON-RAPE)',
    801: 'SEX OFFENSES (NON-RAPE)',
    802: 'SEX OFFENSES (NON-RAPE)',
    803: 'SEX OFFENSES (NON-RAPE)',
    900: 'OTHER OFFENSES',
    901: 'OTHER OFFENSES',
    902: 'OTHER OFFENSES',
    903: 'OTHER OFFENSES',
    904: 'OTHER OFFENSES',
    905: 'OTHER OFFENSES',
    906: 'OTHER OFFENSES',
    907: 'OTHER OFFENSES'
}

# Crear nueva columna 'crime_ucr' basada en el mapping
df['crime_ucr'] = df['crime_code'].map(ucr_mapping).fillna('UNKNOWN')

# Si quieres ver los resultados
print(df[['crime_code', 'crime_desc', 'crime_ucr']].head())


   crime_code                                crime_desc            crime_ucr
0         510                          VEHICLE - STOLEN  MOTOR VEHICLE THEFT
1         330                     BURGLARY FROM VEHICLE   THEFT FROM VEHICLE
2         480                             BIKE - STOLEN          OTHER THEFT
3         343  SHOPLIFTING-GRAND THEFT ($950.01 & OVER)          OTHER THEFT
4         354                         THEFT OF IDENTITY              UNKNOWN


In [143]:
df.columns

Index(['id_report', 'date_report', 'date_ocurrance', 'time_ocurrance',
       'area_code', 'area_name', 'district', 'crime_severity', 'crime_code',
       'crime_desc', 'crime_mo', 'age_victim', 'sex_victim', 'descent_victim',
       'loc_code', 'loc_desc', 'weapon_code', 'weapon_desc',
       'case_status_code', 'case_status_desc', 'crime_code_1', 'crime_code_2',
       'crime_code_3', 'crime_code_4', 'location', 'cross_street', 'latitude',
       'longitude', 'area_valid', 'crime_ucr'],
      dtype='object')

## 4. Investigación de Sexo.

In [144]:
# 1. Primero, corregimos H -> M
df['sex_victim'] = df['sex_victim'].replace('H', 'M')

# 2. Definimos la lista válida final
valores_validos_sexo = ['-', 'F', 'M', 'X']

# 3. Comprobamos de nuevo si queda todo limpio
print("Valores únicos en sex_victim después de la corrección:")
print(df['sex_victim'].unique())

# 4. Opcional: identificar si queda algo inválido
valores_invalidos = df[~df['sex_victim'].isin(valores_validos_sexo)]

print(f"\nCantidad de registros con sex_victim inválido después de corregir: {len(valores_invalidos)}")


Valores únicos en sex_victim después de la corrección:
['M' 'X' 'F' nan '-']

Cantidad de registros con sex_victim inválido después de corregir: 139575


In [145]:
# Reemplazar nulos por '-'
df['sex_victim'] = df['sex_victim'].fillna('-')

# Volver a verificar
print("Valores únicos en sex_victim después de rellenar NaN:")
print(df['sex_victim'].unique())

Valores únicos en sex_victim después de rellenar NaN:
['M' 'X' 'F' '-']


In [146]:
# 1. Conteo de sex_victim
conteo_sexo = df['sex_victim'].value_counts()

# 2. Porcentaje de sex_victim
porcentaje_sexo = df['sex_victim'].value_counts(normalize=True) * 100

# 3. Mostrar resultados
resultado = pd.DataFrame({
    'Cantidad': conteo_sexo,
    'Porcentaje (%)': porcentaje_sexo.round(2)
})

print(resultado)

            Cantidad  Porcentaje (%)
sex_victim                          
M             398060           40.35
F             353688           35.85
-             139576           14.15
X              95176            9.65


## 5. Origen

In [147]:
# 1. Definimos los valores válidos
valores_validos_descent = ['-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 
                           'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z']

# 2. Verificamos los valores únicos en el dataset
print("Valores únicos en descent_victim:")
print(df['descent_victim'].unique())

# 3. Detectamos valores inválidos
valores_invalidos = df[~df['descent_victim'].isin(valores_validos_descent)]

# 4. Mostramos las filas con descent_victim inválido
print("\nFilas con valores inválidos en descent_victim:")
print(valores_invalidos[['id_report', 'descent_victim']])

# 5. Cantidad de registros inválidos
print(f"\nCantidad de registros con descent_victim inválido: {len(valores_invalidos)}")

Valores únicos en descent_victim:
['O' 'X' 'H' 'B' 'W' nan 'A' 'K' 'C' 'J' 'F' 'I' 'V' 'S' 'P' 'Z' 'G' 'U'
 'D' 'L' '-']

Filas con valores inválidos en descent_victim:
        id_report descent_victim
13      221008844            NaN
23      200412582            NaN
26      222106031            NaN
27      200209713            NaN
33      221404806            NaN
...           ...            ...
986479  241107019            NaN
986482  241406728            NaN
986489  241513941            NaN
986496  240710284            NaN
986499  240910892            NaN

[139586 rows x 2 columns]

Cantidad de registros con descent_victim inválido: 139586


In [148]:
# Reemplazar NaN por '-'
df['descent_victim'] = df['descent_victim'].fillna('-')

In [149]:
# 1. Definir los valores válidos de razas
valores_validos_descent = ['-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 
                           'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z']

# 2. Rellenar nulos si es necesario
df['descent_victim'] = df['descent_victim'].fillna('-')

# 3. Conteo de víctimas por raza
conteo_raza = df['descent_victim'].value_counts()

# 4. Porcentaje por raza
porcentaje_raza = df['descent_victim'].value_counts(normalize=True) * 100

# 5. Crear el DataFrame resultado
resultado_raza = pd.DataFrame({
    'Cantidad': conteo_raza,
    'Porcentaje (%)': porcentaje_raza.round(2)
})

# 6. Mostrar resultados
print(resultado_raza)

                Cantidad  Porcentaje (%)
descent_victim                          
H                 293088           29.71
W                 197773           20.05
-                 139588           14.15
B                 134449           13.63
X                 103966           10.54
O                  76983            7.80
A                  21081            2.14
K                   5720            0.58
F                   4592            0.47
C                   4330            0.44
J                   1506            0.15
V                   1128            0.11
I                    980            0.10
Z                    537            0.05
P                    279            0.03
U                    212            0.02
D                     88            0.01
L                     73            0.01
G                     72            0.01
S                     55            0.01



## 6. Localización del suceso

In [150]:
mapping_loc_code_desc = {
    101.0: 'STREET',
    102.0: 'BUS STOP',
    103.0: 'PARKING LOT',
    104.0: 'DRIVEWAY',
    105.0: 'SIDEWALK',
    201.0: 'SINGLE FAMILY DWELLING',
    202.0: 'MULTI-FAMILY DWELLING (APARTMENT)',
    203.0: 'DUPLEX/TRIPLEX',
    204.0: 'ROOMING HOUSE',
    301.0: 'GOVERNMENT BUILDING',
    302.0: 'POLICE FACILITY',
    303.0: 'JAIL/PRISON',
    401.0: 'RESTAURANT',
    402.0: 'BAR/NIGHTCLUB',
    501.0: 'HOSPITAL/MEDICAL FACILITY',
    502.0: 'NURSING HOME',
    601.0: 'HOTEL/MOTEL',
    602.0: 'MOTOR HOTEL',
    701.0: 'SCHOOL, PUBLIC',
    702.0: 'SCHOOL, PRIVATE',
    801.0: 'BANK',
    802.0: 'ATM',
    901.0: 'MARKET',
    902.0: 'CONVENIENCE STORE',
    903.0: 'DEPARTMENT STORE',
    904.0: 'SHOPPING MALL',
    905.0: 'GAS STATION',
    906.0: 'AUTO DEALERSHIP',
    907.0: 'PARK',
    908.0: 'PLAYGROUND',
    909.0: 'GYM/FITNESS CENTER',
    910.0: 'THEATER',
    911.0: 'MUSEUM',
    912.0: 'LIBRARY',
    913.0: 'CHURCH/SYNAGOGUE/TEMPLE',
    914.0: 'WAREHOUSE',
    915.0: 'FACTORY/PLANT',
    916.0: 'CONSTRUCTION SITE',
    917.0: 'VACANT LOT',
    918.0: 'ABANDONED BUILDING',
    919.0: 'BUSINESS OFFICE',
    920.0: 'YARD (RESIDENTIAL/BUSINESS)',
    976.0: 'OTHER LOCATION'
}


In [151]:
# Mapear el loc_desc estándar según loc_code
df['loc_desc'] = df['loc_code'].map(mapping_loc_code_desc).fillna('UNKNOWN')

In [152]:
# 1. Contar crímenes por localización (loc_desc)
conteo_localizaciones = df['loc_desc'].value_counts()

# 2. Sacar porcentaje
porcentaje_localizaciones = (conteo_localizaciones / conteo_localizaciones.sum()) * 100

# 3. Juntar en un solo DataFrame
resultado_localizaciones = pd.DataFrame({
    'Cantidad': conteo_localizaciones,
    'Porcentaje (%)': porcentaje_localizaciones.round(2)
})

# 4. Tomar solo el TOP 10
top_10_localizaciones = resultado_localizaciones.head(10)

# 5. Mostrar
print(top_10_localizaciones)



                           Cantidad  Porcentaje (%)
loc_desc                                           
UNKNOWN                      303434           30.76
STREET                       254978           25.85
HOSPITAL/MEDICAL FACILITY    162104           16.43
NURSING HOME                 117577           11.92
DUPLEX/TRIPLEX                46810            4.75
BUS STOP                      40553            4.11
DRIVEWAY                      15770            1.60
BAR/NIGHTCLUB                  7702            0.78
PARKING LOT                    6943            0.70
GOVERNMENT BUILDING            5531            0.56


# 7. Investigación sobre las armas

In [153]:
mapping_weapon_code_desc = {
    100.0: 'HAND GUN',
    101.0: 'RIFLE',
    102.0: 'SHOTGUN',
    103.0: 'ASSAULT WEAPON (AK47, UZI, etc.)',
    104.0: 'OTHER FIREARM',
    105.0: 'SEMI-AUTOMATIC HANDGUN',
    106.0: 'SEMI-AUTOMATIC RIFLE',
    200.0: 'KNIFE/SHARP INSTRUMENT',
    201.0: 'RAZOR',
    202.0: 'SCISSORS',
    203.0: 'ICE PICK',
    204.0: 'SWITCH BLADE',
    205.0: 'OTHER CUTTING INSTRUMENT',
    206.0: 'SWORD/MACHETE',
    300.0: 'BLUNT OBJECT (CLUB, BAT, etc.)',
    301.0: 'ROCK/BRICK',
    302.0: 'PIPE/METAL BAR',
    303.0: 'STICK/POLE',
    400.0: 'PERSONAL WEAPONS (HANDS, FISTS, FEET, ETC.)',
    500.0: 'OTHER WEAPON',
    501.0: 'BOTTLE',
    502.0: 'CHAIN',
    503.0: 'ROPE/STRING/SHOELACE',
    504.0: 'CHEMICAL SPRAY (MACE/PEPPER SPRAY)',
    505.0: 'FIRE (ARSON)',
    506.0: 'EXPLOSIVES',
    507.0: 'VEHICLE',
    508.0: 'AXE',
    509.0: 'SCREWDRIVER',
    510.0: 'UNKNOWN WEAPON',
    700.0: 'SIMULATED WEAPON (FAKE GUN, ETC.)',
    800.0: 'VERBAL THREAT',
    900.0: 'OTHER DANGEROUS WEAPON',
}

In [154]:
# 1. Aseguramos que 'weapon_code' esté correctamente mapeado
df['weapon_desc'] = df['weapon_code'].map(mapping_weapon_code_desc).fillna('UNKNOWN')

# 2. Conteo de armas por descripción
conteo_armas = df['weapon_desc'].value_counts()

# 3. Porcentaje de armas
porcentaje_armas = (conteo_armas / conteo_armas.sum()) * 100

# 4. Crear el DataFrame final
resultado_armas = pd.DataFrame({
    'Cantidad': conteo_armas,
    'Porcentaje (%)': porcentaje_armas.round(2)
})

# 5. Tomar el TOP 10
top_10_armas = resultado_armas.head(10)

# 6. Mostrar
print(top_10_armas)



                                             Cantidad  Porcentaje (%)
weapon_desc                                                          
UNKNOWN                                        730381           74.04
PERSONAL WEAPONS (HANDS, FISTS, FEET, ETC.)    174484           17.69
OTHER WEAPON                                    36010            3.65
SHOTGUN                                         20123            2.04
KNIFE/SHARP INSTRUMENT                           6826            0.69
SEMI-AUTOMATIC RIFLE                             6573            0.67
SWITCH BLADE                                     2260            0.23
OTHER CUTTING INSTRUMENT                         1919            0.19
RAZOR                                            1697            0.17
PIPE/METAL BAR                                   1385            0.14



## 8. Investigación sobre Modus Operandi

## 9. Investigación - Case Status Code/Description

In [155]:
unique_status = df['case_status_code'].unique()
print("Valores únicos en la columna 'Status':")
print(unique_status)

non_standard_status = df[df['case_status_code'].isin(['AO', 'JO'])]
result = non_standard_status[['case_status_code', 'case_status_desc']].head(5)
print(result)

Valores únicos en la columna 'Status':
['AA' 'IC' 'AO' 'JA' 'JO' 'CC' nan]
   case_status_code case_status_desc
21               AO      Adult Other
25               AO      Adult Other
35               AO      Adult Other
81               AO      Adult Other
86               AO      Adult Other


In [156]:
status_not_null = df['case_status_code'].notna()
total_status = status_not_null.sum()

non_standard_count = df['case_status_code'].isin(['AO', 'JO']).sum()
percentage = (non_standard_count / total_status) * 100

print(f"Cantidad total de valores no estándar ('AO' y 'JO'): {non_standard_count}")
print(f"Porcentaje respecto al total de valores definidos en 'Status': {percentage:.2f}%")

Cantidad total de valores no estándar ('AO' y 'JO'): 109569
Porcentaje respecto al total de valores definidos en 'Status': 11.11%


In [157]:
null_count = df['case_status_code'].isna().sum()
total_rows = len(df)
null_percentage = (null_count / total_rows) * 100

print(f"Cantidad de valores nulos en 'Status': {null_count}")
print(f"Porcentaje respecto al total de filas: {null_percentage:.2f}%")

Cantidad de valores nulos en 'Status': 1
Porcentaje respecto al total de filas: 0.00%


In [158]:
null_count_desc = df['case_status_desc'].isna().sum()
total_rows = len(df)
null_percentage_desc = (null_count_desc / total_rows) * 100

print(f"Cantidad de valores nulos en 'Status Desc': {null_count_desc}")
print(f"Porcentaje respecto al total de filas: {null_percentage_desc:.2f}%")

Cantidad de valores nulos en 'Status Desc': 0
Porcentaje respecto al total de filas: 0.00%


In [159]:
status_null_rows = df[df['case_status_code'].isna()]
print(status_null_rows[['case_status_desc']])

       case_status_desc
881136              UNK



## Valores numericos (Aaron y Javi)


### id_report

In [160]:
df['id_report'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: id_report
Non-Null Count   Dtype
--------------   -----
986500 non-null  int64
dtypes: int64(1)
memory usage: 7.5 MB


In [161]:
df.iloc[:, :5].head(10)

,id_report,date_report,date_ocurrance,time_ocurrance,area_code
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6
5,231808869,04/04/2023 12:00:00 AM,12/01/2020 12:00:00 AM,2300,1
6,230110144,04/04/2023 12:00:00 AM,07/03/2020 12:00:00 AM,900,1
7,220314085,07/22/2022 12:00:00 AM,05/12/2020 12:00:00 AM,1110,3
8,231309864,04/28/2023 12:00:00 AM,12/09/2020 12:00:00 AM,1400,1
9,211904005,12/31/2020 12:00:00 AM,12/31/2020 12:00:00 AM,1220,1


In [162]:
df.loc[df['id_report']<9999999]

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude,area_valid,crime_ucr
39890,817,09/20/2020 12:00:00 AM,09/19/2020 12:00:00 AM,1700,1,77th Street,1777,1,510,VEHICLE - STOLEN,...,510.0,NaN,NaN,NaN,9100 RUBIO AV,NaN,34.2367,-118.4955,True,MOTOR VEHICLE THEFT
268033,2113,06/21/2021 12:00:00 AM,04/05/2021 12:00:00 AM,1835,1,77th Street,1385,1,510,VEHICLE - STOLEN,...,510.0,NaN,NaN,NaN,800 E GAGE AV,NaN,33.9820,-118.2622,True,MOTOR VEHICLE THEFT
581052,2203,12/31/2022 12:00:00 AM,12/19/2022 12:00:00 AM,1930,3,Devonshire,356,1,522,"VEHICLE, STOLEN - OTHER (MOTORIZED SCOOTERS, B...",...,522.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,True,UNKNOWN
734486,2315,07/23/2023 12:00:00 AM,07/22/2023 12:00:00 AM,1800,1,77th Street,1566,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,420.0,NaN,NaN,NaN,4400 ENSIGN AV,NaN,34.1504,-118.3712,True,THEFT FROM VEHICLE
876571,2401,01/10/2024 12:00:00 AM,12/21/2023 12:00:00 AM,1930,1,77th Street,1416,1,510,VEHICLE - STOLEN,...,510.0,NaN,NaN,NaN,3400 CABRILLO BL,NaN,34.0099,-118.4453,True,MOTOR VEHICLE THEFT


### date_report

In [163]:
df['date_report']=pd.to_datetime(df['date_report'])

C:\Users\aaron\AppData\Local\Temp\ipykernel_22420\240740369.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_report']=pd.to_datetime(df['date_report'])


In [164]:
df['date_report'].head()

0   2020-03-01
1   2020-02-09
2   2020-11-11
3   2023-05-10
4   2022-08-18
Name: date_report, dtype: datetime64[ns]

In [165]:
df['date_report'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: date_report
Non-Null Count   Dtype         
--------------   -----         
986500 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.5 MB


In [166]:
df['month_report'] = df['date_report'].dt.month #Nueva columna solo con el mes del reporte


In [167]:
df['year_report'] = df['date_report'].dt.year   #Nueva columna solo con el año del reporte

In [168]:
df['year_month_report'] = df['date_report'].dt.strftime('%Y-%m')    #Nueva columna solo con el año y mes del reporte


In [169]:
df.iloc[:,20:].head()

,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude,area_valid,crime_ucr,month_report,year_report,year_month_report
0,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,True,MOTOR VEHICLE THEFT,3,2020,2020-03
1,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,True,THEFT FROM VEHICLE,2,2020,2020-02
2,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,True,OTHER THEFT,11,2020,2020-11
3,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,True,OTHER THEFT,5,2023,2023-05
4,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,True,UNKNOWN,8,2022,2022-08


### date_ocurrance

In [170]:
df['date_ocurrance']=pd.to_datetime(df['date_ocurrance'])

C:\Users\aaron\AppData\Local\Temp\ipykernel_22420\779857604.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_ocurrance']=pd.to_datetime(df['date_ocurrance'])


In [171]:
df['date_ocurrance'].head()

0   2020-03-01
1   2020-02-08
2   2020-11-04
3   2020-03-10
4   2020-08-17
Name: date_ocurrance, dtype: datetime64[ns]

In [172]:
df['date_ocurrance'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: date_ocurrance
Non-Null Count   Dtype         
--------------   -----         
986500 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 7.5 MB


In [173]:
df['month_ocurrance'] = df['date_ocurrance'].dt.month   #Nueva columna solo con el mes de la aparicion

In [174]:
df['year_ocurrance'] = df['date_ocurrance'].dt.year ##Nueva columna solo con el año de la aparicion

In [175]:
df['year_month_ocurrance'] = df['date_ocurrance'].dt.strftime('%Y-%m')  ##Nueva columna solo con el año y mes de la aparicion

### time_ocurrance

In [176]:
df['time_ocurrance'] = df['time_ocurrance'].astype(str).str.zfill(4) #Asi pongo 4 cifras en la columna

In [177]:
df['time_ocurrance'] = df['time_ocurrance'].str[:-2] + ":" + df['time_ocurrance'].str[-2:] # Aqui converti a string y luego lo separe lo 2 primero numeros, luego : y luego los 2 ultimos

In [178]:
df['time_ocurrance'] = pd.to_datetime(df['time_ocurrance'], format ='%H:%M').dt.time

In [179]:
df.head(5)

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,latitude,longitude,area_valid,crime_ucr,month_report,year_report,year_month_report,month_ocurrance,year_ocurrance,year_month_ocurrance
0,190326475,2020-03-01,2020-03-01,21:30:00,7,Hollywood,784,1,510,VEHICLE - STOLEN,...,34.0375,-118.3506,True,MOTOR VEHICLE THEFT,3,2020,2020-03,3,2020,2020-03
1,200106753,2020-02-09,2020-02-08,18:00:00,1,77th Street,182,1,330,BURGLARY FROM VEHICLE,...,34.0444,-118.2628,True,THEFT FROM VEHICLE,2,2020,2020-02,2,2020,2020-02
2,200320258,2020-11-11,2020-11-04,17:00:00,3,Devonshire,356,1,480,BIKE - STOLEN,...,34.0210,-118.3002,True,OTHER THEFT,11,2020,2020-11,11,2020,2020-11
3,200907217,2023-05-10,2020-03-10,20:37:00,9,N Hollywood,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,34.1576,-118.4387,True,OTHER THEFT,5,2023,2023-05,3,2020,2020-03
4,220614831,2022-08-18,2020-08-17,12:00:00,6,Hollenbeck,666,2,354,THEFT OF IDENTITY,...,34.0944,-118.3277,True,UNKNOWN,8,2022,2022-08,8,2020,2020-08


### age_victim

Mostrar la información de los no nulos de la columna age_victim

In [180]:
df['age_victim'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 986500 entries, 0 to 986499
Series name: age_victim
Non-Null Count   Dtype
--------------   -----
986500 non-null  int64
dtypes: int64(1)
memory usage: 7.5 MB


Mostrar la información de los valores únicos de la columna

In [181]:
if 'age_victim' in df.columns:
    print(sorted(df['age_victim'].dropna().unique()))
    print("\n")
else:
    print("Column 'age_victim' not found in the dataframe.\n")

[np.int64(-4), np.int64(-3), np.int64(-2), np.int64(-1), np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34), np.int64(35), np.int64(36), np.int64(37), np.int64(38), np.int64(39), np.int64(40), np.int64(41), np.int64(42), np.int64(43), np.int64(44), np.int64(45), np.int64(46), np.int64(47), np.int64(48), np.int64(49), np.int64(50), np.int64(51), np.int64(52), np.int64(53), np.int64(54), np.int64(55), np.int64(56), np.int64(57), np.int64(58), np.int64(59), np.int64(60), np.int64(61), np.int64(62), np.int64(63), np.int64(64), np.int64(65), np.int64(66), np.int64(67), np.int64(68), 

Mostrar la información de los valores negativos

In [183]:
df['age_victim'].loc[df['age_victim']<0]

143      -2
375      -2
581      -1
595      -1
618      -1
         ..
636182   -1
654233   -1
681376   -1
728608   -1
828481   -1
Name: age_victim, Length: 132, dtype: int64

In [184]:
(df['age_victim']<1).sum()

np.int64(261553)

In [185]:
df.columns

Index(['id_report', 'date_report', 'date_ocurrance', 'time_ocurrance',
       'area_code', 'area_name', 'district', 'crime_severity', 'crime_code',
       'crime_desc', 'crime_mo', 'age_victim', 'sex_victim', 'descent_victim',
       'loc_code', 'loc_desc', 'weapon_code', 'weapon_desc',
       'case_status_code', 'case_status_desc', 'crime_code_1', 'crime_code_2',
       'crime_code_3', 'crime_code_4', 'location', 'cross_street', 'latitude',
       'longitude', 'area_valid', 'crime_ucr', 'month_report', 'year_report',
       'year_month_report', 'month_ocurrance', 'year_ocurrance',
       'year_month_ocurrance'],
      dtype='object')

In [189]:
age_area = df.groupby('area_name')['age_victim'].agg(lambda x: x.median())

df.loc[df['age_victim'] <= 0, 'age_victim'] = df.loc[df['age_victim'] <= 0, 'area_name'].map(age_area)

### location

In [191]:
df['location'].unique()

array(['1900 S  LONGWOOD                     AV',
       '1000 S  FLOWER                       ST',
       '1400 W  37TH                         ST', ...,
       '9500    LA CIENEGA                   BL',
       '1800 E  MARTIN LUTHER KING JR        BL',
       '12200    ART                          ST'],
      shape=(66322,), dtype=object)

In [192]:
df['location'] = df['location'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))    #la funcion re.sub es para reemplazar (lo que quieres reemplazar, por lo que lo vas a reemplazar)

In [193]:
df['location'] = df['location'].apply(str.strip).apply(str.lower)    # Para quitar espacios y ponerlo en minuscula

In [194]:
df.head()

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,latitude,longitude,area_valid,crime_ucr,month_report,year_report,year_month_report,month_ocurrance,year_ocurrance,year_month_ocurrance
0,190326475,2020-03-01,2020-03-01,21:30:00,7,Hollywood,784,1,510,VEHICLE - STOLEN,...,34.0375,-118.3506,True,MOTOR VEHICLE THEFT,3,2020,2020-03,3,2020,2020-03
1,200106753,2020-02-09,2020-02-08,18:00:00,1,77th Street,182,1,330,BURGLARY FROM VEHICLE,...,34.0444,-118.2628,True,THEFT FROM VEHICLE,2,2020,2020-02,2,2020,2020-02
2,200320258,2020-11-11,2020-11-04,17:00:00,3,Devonshire,356,1,480,BIKE - STOLEN,...,34.0210,-118.3002,True,OTHER THEFT,11,2020,2020-11,11,2020,2020-11
3,200907217,2023-05-10,2020-03-10,20:37:00,9,N Hollywood,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,34.1576,-118.4387,True,OTHER THEFT,5,2023,2023-05,3,2020,2020-03
4,220614831,2022-08-18,2020-08-17,12:00:00,6,Hollenbeck,666,2,354,THEFT OF IDENTITY,...,34.0944,-118.3277,True,UNKNOWN,8,2022,2022-08,8,2020,2020-08


### crime_code_2

In [196]:
df['crime_code_2'] = df['crime_code_2'].fillna(0)

In [197]:
df['crime_code_2'].head(10)

0    998.0
1    998.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
Name: crime_code_2, dtype: float64

In [198]:
df['crime_code_2'] = df['crime_code_2'].astype(int)

### cross_street

In [199]:
df['cross_street'].unique()

array([nan, 'VANOWEN                      AV', 'INGLEWOOD', ...,
       'CHARLES E YOUNG              DR',
       'LUCERNE                      ST', '222ND'],
      shape=(10338,), dtype=object)

In [200]:
df['cross_street'] = df['cross_street'].fillna('Unknow')

In [201]:
df['cross_street'] = df['cross_street'].astype(str)

In [202]:
df['cross_street'] = df['cross_street'].apply(str.strip).apply(str.lower)    # Para quitar espacios y ponerlo en minuscula

In [205]:
df.iloc[:,20:].tail()

,crime_code_1,crime_code_2,crime_code_3,crime_code_4,location,cross_street,latitude,longitude,area_valid,crime_ucr,month_report,year_report,year_month_report,month_ocurrance,year_ocurrance,year_month_ocurrance
986495,341.0,0,NaN,NaN,3700 wilshire bl,unknow,34.0617,-118.3066,True,OTHER THEFT,8,2024,2024-08,8,2024,2024-08
986496,510.0,0,NaN,NaN,4000 w 23rd st,unknow,34.0362,-118.3284,True,MOTOR VEHICLE THEFT,7,2024,2024-07,7,2024,2024-07
986497,745.0,0,NaN,NaN,1300 w sunset bl,unknow,34.0685,-118.2460,True,UNKNOWN,1,2024,2024-01,1,2024,2024-01
986498,230.0,0,NaN,NaN,flower st,jefferson bl,34.0215,-118.2868,True,AGGRAVATED ASSAULT,4,2024,2024-04,4,2024,2024-04
986499,510.0,0,NaN,NaN,6900 vesper av,unknow,34.1961,-118.4510,True,MOTOR VEHICLE THEFT,8,2024,2024-08,8,2024,2024-08


### latitude

In [206]:
df['latitude'].describe()

count    986500.000000
mean         33.996033
std           1.633543
min           0.000000
25%          34.014600
50%          34.058900
75%          34.164900
max          34.334300
Name: latitude, dtype: float64

In [207]:
# Contar cuantas veces aparece el valor 0 en la columna latitude
print(f" La cantidad de valores 0 en la columna 'latitude' es de: {(df['latitude'] == 0).sum()} y el porcentaje total es de {((df['latitude'] == 0).sum())/df.shape[0]*100:.2f} %")

 La cantidad de valores 0 en la columna 'latitude' es de: 2262 y el porcentaje total es de 0.23 %


In [72]:
print(f"Las filas que coinciden son: {((df['latitude'] == 0) & (df['longitude'] == 0)).sum()}")



Las filas que coinciden son: 2262


In [208]:
((df['latitude']<10)).sum()

np.int64(2262)

In [209]:
df.loc[(df['latitude']<10)].head(5)

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,latitude,longitude,area_valid,crime_ucr,month_report,year_report,year_month_report,month_ocurrance,year_ocurrance,year_month_ocurrance
559,211410751,2021-04-30,2020-06-12,09:30:00,1,77th Street,1438,2,354,THEFT OF IDENTITY,...,0.0,0.0,True,UNKNOWN,4,2021,2021-04,6,2020,2020-06
2268,200416028,2020-12-17,2020-12-17,15:25:00,4,Foothill,473,2,900,VIOLATION OF COURT ORDER,...,0.0,0.0,True,OTHER OFFENSES,12,2020,2020-12,12,2020,2020-12
2637,201517780,2020-11-20,2020-11-20,18:49:00,1,77th Street,1555,1,648,ARSON,...,0.0,0.0,True,UNKNOWN,11,2020,2020-11,11,2020,2020-11
2922,200817064,2020-12-01,2020-12-01,13:40:00,8,Mission,882,1,440,THEFT PLAIN - PETTY ($950 & UNDER),...,0.0,0.0,True,OTHER THEFT,12,2020,2020-12,12,2020,2020-12
3095,201410528,2020-05-06,2020-05-06,07:40:00,1,77th Street,1494,2,888,TRESPASSING,...,0.0,0.0,True,UNKNOWN,5,2020,2020-05,5,2020,2020-05


In [216]:
location_lat_mode = df.groupby('location')['latitude'].agg(lambda x: x.mode().iloc[0])  #Metodo para agrupar por localizacion e implementar la moda

df.loc[df['latitude'] == 0, 'latitude'] = df.loc[df['latitude'] == 0, 'location'].map(location_lat_mode)

In [217]:
df.drop(df[df['latitude'] <10].index, inplace=True)



### longitude

In [218]:
print(f" La cantidad de valores 0 en la columna 'longitude' es de: {(df['longitude'] == 0).sum()} y el porcentaje total es de {((df['longitude'] == 0).sum())/df.shape[0]*100:.2f} %")

 La cantidad de valores 0 en la columna 'longitude' es de: 2 y el porcentaje total es de 0.00 %


In [219]:
((df['longitude']>-110)).sum()

np.int64(2)

In [220]:
location_lon_mode = df.groupby('location')['longitude'].agg(lambda x: x.mode().iloc[0])

df.loc[df['longitude'] == 0, 'longitude'] = df.loc[df['longitude'] == 0, 'location'].map(location_lon_mode)

In [221]:
df.drop(df[df['longitude'] >-110].index, inplace=True)

# Modificaciones finales

In [222]:
porcentaje_nulos = df.isna().mean() * 100
print(porcentaje_nulos)


id_report                0.000000
date_report              0.000000
date_ocurrance           0.000000
time_ocurrance           0.000000
area_code                0.000000
area_name                0.000000
district                 0.000000
crime_severity           0.000000
crime_code               0.000000
crime_desc               0.000000
crime_mo                14.844353
age_victim               0.000000
sex_victim               0.000000
descent_victim           0.000000
loc_code                 0.001419
loc_desc                 0.000000
weapon_code             66.916626
weapon_desc              0.000000
case_status_code         0.000101
case_status_desc         0.000000
crime_code_1             0.001115
crime_code_2             0.000000
crime_code_3            99.765822
crime_code_4            99.993512
location                 0.000000
cross_street             0.000000
latitude                 0.000000
longitude                0.000000
area_valid               0.000000
crime_ucr     

In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 986429 entries, 0 to 986499
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id_report             986429 non-null  int64         
 1   date_report           986429 non-null  datetime64[ns]
 2   date_ocurrance        986429 non-null  datetime64[ns]
 3   time_ocurrance        986429 non-null  object        
 4   area_code             986429 non-null  int64         
 5   area_name             986429 non-null  object        
 6   district              986429 non-null  object        
 7   crime_severity        986429 non-null  int64         
 8   crime_code            986429 non-null  int64         
 9   crime_desc            986429 non-null  object        
 10  crime_mo              840000 non-null  object        
 11  age_victim            986429 non-null  int64         
 12  sex_victim            986429 non-null  object        
 13  desc

## Columnas a eliminar

In [224]:
columns_drop = ['crime_code_4','crime_code_3','crime_code_1',]

In [225]:
df = df.drop(columns_drop, axis=1)


## Filas a eliminar

In [226]:
features_null = ['loc_desc','loc_code','case_status_code']

In [227]:
df = df.dropna(subset=features_null)


In [228]:
df.head()

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,latitude,longitude,area_valid,crime_ucr,month_report,year_report,year_month_report,month_ocurrance,year_ocurrance,year_month_ocurrance
0,190326475,2020-03-01,2020-03-01,21:30:00,7,Hollywood,784,1,510,VEHICLE - STOLEN,...,34.0375,-118.3506,True,MOTOR VEHICLE THEFT,3,2020,2020-03,3,2020,2020-03
1,200106753,2020-02-09,2020-02-08,18:00:00,1,77th Street,182,1,330,BURGLARY FROM VEHICLE,...,34.0444,-118.2628,True,THEFT FROM VEHICLE,2,2020,2020-02,2,2020,2020-02
2,200320258,2020-11-11,2020-11-04,17:00:00,3,Devonshire,356,1,480,BIKE - STOLEN,...,34.0210,-118.3002,True,OTHER THEFT,11,2020,2020-11,11,2020,2020-11
3,200907217,2023-05-10,2020-03-10,20:37:00,9,N Hollywood,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,34.1576,-118.4387,True,OTHER THEFT,5,2023,2023-05,3,2020,2020-03
4,220614831,2022-08-18,2020-08-17,12:00:00,6,Hollenbeck,666,2,354,THEFT OF IDENTITY,...,34.0944,-118.3277,True,UNKNOWN,8,2022,2022-08,8,2020,2020-08


In [229]:
df['district'] = df['district'].astype(int)

## Pasar a minuscula todas las variables object para estandarizar todo

In [230]:
for columna in df.select_dtypes(include='object').columns:
    if columna != 'time_ocurrance':
        df[columna] = df[columna].str.lower()



In [231]:
df['crime_severity'] = df['crime_severity'].map({1: 'grave', 2: 'menos grave'})


In [237]:
df = df.drop_duplicates()

In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 986415 entries, 0 to 986499
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id_report             986415 non-null  int64         
 1   date_report           986415 non-null  datetime64[ns]
 2   date_ocurrance        986415 non-null  datetime64[ns]
 3   time_ocurrance        986415 non-null  object        
 4   area_code             986415 non-null  int64         
 5   area_name             986415 non-null  object        
 6   district              986415 non-null  int64         
 7   crime_severity        986415 non-null  object        
 8   crime_code            986415 non-null  int64         
 9   crime_desc            986415 non-null  object        
 10  crime_mo              840000 non-null  object        
 11  age_victim            986415 non-null  int64         
 12  sex_victim            986415 non-null  object        
 13  desc

# EXPORTAR CSV

In [233]:
df.head()

,id_report,date_report,date_ocurrance,time_ocurrance,area_code,area_name,district,crime_severity,crime_code,crime_desc,...,latitude,longitude,area_valid,crime_ucr,month_report,year_report,year_month_report,month_ocurrance,year_ocurrance,year_month_ocurrance
0,190326475,2020-03-01,2020-03-01,21:30:00,7,hollywood,784,grave,510,vehicle - stolen,...,34.0375,-118.3506,True,motor vehicle theft,3,2020,2020-03,3,2020,2020-03
1,200106753,2020-02-09,2020-02-08,18:00:00,1,77th street,182,grave,330,burglary from vehicle,...,34.0444,-118.2628,True,theft from vehicle,2,2020,2020-02,2,2020,2020-02
2,200320258,2020-11-11,2020-11-04,17:00:00,3,devonshire,356,grave,480,bike - stolen,...,34.0210,-118.3002,True,other theft,11,2020,2020-11,11,2020,2020-11
3,200907217,2023-05-10,2020-03-10,20:37:00,9,n hollywood,964,grave,343,shoplifting-grand theft ($950.01 & over),...,34.1576,-118.4387,True,other theft,5,2023,2023-05,3,2020,2020-03
4,220614831,2022-08-18,2020-08-17,12:00:00,6,hollenbeck,666,menos grave,354,theft of identity,...,34.0944,-118.3277,True,unknown,8,2022,2022-08,8,2020,2020-08


In [236]:
df.to_csv('Data_Crime_Cleaning.csv', index=False)